In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from imutils import contours
from PIL.Image import Resampling
from PIL import Image
import PIL

In [ ]:
def show(img, size=75):
    plt.figure(figsize=(size, size))

    if len(img.shape) == 2:  # if grayscale, set cmap
        plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    else:  # if 3 channels, change from BGR to RGB
        plt.imshow(img[..., ::-1])

    plt.show()

def get_first_frame(videofile):
    vidcap = cv2.VideoCapture(videofile)
    success, image = vidcap.read()
    if success:
        cv2.imwrite("book_video_first_frame.jpg", image)  # save frame as JPEG file


def get_keypoints(img):
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(img,None)
    img_keypoints=cv2.drawKeypoints(img,kp,img)
    return kp, des


def get_correspondences(img1, img2):
    kp1, des1 = get_keypoints(img1)
    kp2, des2 = get_keypoints(img2)

    # BFMatcher with default params
    bf = cv2.BFMatcher()

    # des1 = queryDescriptor, des2 = trainDescriptor
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    final_matches = []
    for m, n in matches:
        if m.distance < 0.235 * n.distance:
            good.append([m])
            final_matches.append([kp1[m.queryIdx].pt, kp2[m.trainIdx].pt])

    # cv.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    # show(img3)
    print(f"found {len(good)} matches")

    return final_matches, img3


def construct_homography_matrix(matches):
    A = [];
    for match in matches:
        xi = match[0][0]
        yi = match[0][1]

        xj = match[1][0]
        yj = match[1][1]

        A.append([xi, yi, 1, 0, 0, 0, -1 * xj * xi, -1 * xj * yi, -1 * xj])
        A.append([0, 0, 0, xi, yi, 1, -1 * yj * xi, -1 * yj * yi, -1 * yj])

    # print(A)

    u, s, vh = np.linalg.svd(A, full_matrices=True)
    h = vh[-1]
    return np.array(h).reshape((3, 3))


def aspect_ratio(cnt):
   x, y, w, h = cv2.boundingRect(cnt)
   ratio = float(w)/h
   return ratio


def get_contour(image_path):
    image = cv2.imread(image_path)
    original = image.copy()
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    (cnts, _) = contours.sort_contours(cnts, method="left-to-right")

    frame_contour = cnts[0]
    x,y,w,h = cv2.boundingRect(frame_contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (35,255,12), 1)
    Contour_image = original[y:y+h, x:x+w]
    cv2.imwrite('The_Contour.png', Contour_image)

    return frame_contour


def resize(image_path, ratio):
    img = Image.open(image_path)
    hsize = int((float(img.size[1])*float(ratio)))
    wsize = int((float(img.size[0])*float(ratio)))
    resized_image = img.resize((wsize, hsize), Resampling.BICUBIC)
    resized_image.save(image_path[:-4] + '_resized.jpg')

    resized_image



In [ ]:
get_first_frame('book.mov')
book_cover = cv2.imread('cv_cover.jpg')
book_video_first_frame = cv2.imread('book_video_first_frame.jpg')

book_cover = cv2.cvtColor(book_cover, cv2.COLOR_BGR2GRAY)
book_video_first_frame = cv2.cvtColor(book_video_first_frame, cv2.COLOR_BGR2GRAY)

final_matches, img3 = get_correspondences(book_cover, book_video_first_frame)
cv2.imshow('SIFT', img3)
# cv2.waitKey(0)

H = construct_homography_matrix(final_matches) # The homography matrix
# print(H)

img1_out = cv2.warpPerspective(book_cover, H, (book_video_first_frame.shape[1], book_video_first_frame.shape[0]))
cv2.imshow('Warp Perspective', img1_out)
# cv2.waitKey(0)


cartoon_video_first_frame = cv2.imread('cartoon_video_first_frame.jpg')

img2_out = cv2.warpPerspective(cartoon_video_first_frame, H, (book_video_first_frame.shape[1], book_video_first_frame.shape[0]))
cv2.imshow('Warp Perspective', img2_out)
cv2.waitKey()


cv2.imwrite("book_video_first_frame_wrapped.jpg", img1_out)
cv2.imwrite("cartoon_video_first_frame_wrapped.jpg", img2_out)

In [ ]:
book_video_path = 'book_video_first_frame_wrapped.jpg'
book_frame_contour = get_contour(book_video_path)
book_frame_aspect_ratio = aspect_ratio(book_frame_contour)
print("Book Frame Aspect Ratio = " + str(book_frame_aspect_ratio))

cartoon_video_path = 'cartoon_video_first_frame_wrapped.jpg'
cartoon_frame_contour = get_contour(cartoon_video_path)
cartoon_frame_aspect_ratio = aspect_ratio(cartoon_frame_contour)
print("Cartoon Frame Aspect Ratio = " + str(cartoon_frame_aspect_ratio))


resized_image = resize('cartoon_video_first_frame_wrapped.jpg', book_frame_aspect_ratio)